<a href="https://colab.research.google.com/github/timothyow/4am_test/blob/main/category_forecast_PH_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [6]:
from google.colab import files
import os

# Check if the file exists, if not, prompt for upload
excel_file_path = '/content/PH_forecast_dataset.xlsx'
if not os.path.exists(excel_file_path):
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f'User uploaded file "{filename}"')

Saving PH_forecast_dataset.xlsx to PH_forecast_dataset.xlsx
User uploaded file "PH_forecast_dataset.xlsx"


# Task
Analyze the time series data in all tabs of the uploaded Excel file "ok file", where 'Periods' is the date column and all other numeric columns are individual time series. For each time series, compare the performance of the 5 most common time series models (including ARIMA with a specified grid search for parameters d=[0, 1], p=[0, 1, 2, 3, 4], q=[0, 1, 2, 3, 4], seasonal_order=[(0, 0, 0, 12), (0, 1, 0, 12), (1, 0, 1, 12)]) based on MAPE for a 60-period forecast horizon, and then generate and present the best forecast for each time series.

## Load data from all tabs

### Subtask:
Read the Excel file and load data from all available tabs into separate dataframes.


**Reasoning**:
Read the Excel file and load data from all sheets into a dictionary of dataframes.



In [8]:
excel_file_path = '/content/PH_forecast_dataset.xlsx'
excel_file = pd.ExcelFile(excel_file_path)
sheet_names = excel_file.sheet_names
dataframes = {}

for sheet_name in sheet_names:
    dataframes[sheet_name] = excel_file.parse(sheet_name)

for sheet_name, df in dataframes.items():
    print(f"DataFrame from sheet '{sheet_name}':")
    display(df.head())

DataFrame from sheet 'SCM value':


,Periods,CONDENSED MILK,ALASKA MILK,ALASKA | REGULAR,ALASKA | CONDENSADA,COWBELL,LIBERTY,ETIKA DAIRES,LTH FOOD INDUSTRIES,CENTURY PACIFIC FOOD
0,2022-06-01,6.006975e+08,3.367026e+08,4.693034e+07,1.128291e+08,1.428588e+08,2.556323e+07,9.136781e+07,8.997870e+07,1.760445e+07
1,2022-07-01,6.512608e+08,3.497930e+08,5.244049e+07,1.171757e+08,1.456548e+08,2.504313e+07,1.316913e+08,8.684414e+07,1.906383e+07
2,2022-08-01,6.028338e+08,3.406270e+08,4.922567e+07,1.115029e+08,1.442312e+08,2.599094e+07,1.012829e+08,7.932513e+07,1.674502e+07
3,2022-09-01,5.913090e+08,3.361470e+08,4.386226e+07,9.843329e+07,1.558864e+08,2.717121e+07,8.960323e+07,8.028158e+07,1.543614e+07
4,2022-10-01,6.822277e+08,3.684553e+08,5.525348e+07,1.098614e+08,1.657480e+08,2.713870e+07,1.318875e+08,1.017035e+08,1.903207e+07


DataFrame from sheet 'SCM volume':


,Periods,CONDENSED MILK,ALASKA MILK,ALASKA | REGULAR,ALASKA | CONDENSADA,COWBELL,LIBERTY,ETIKA DAIRES,LTH FOOD INDUSTRIES,CENTURY PACIFIC FOOD
0,2022-06-01,4.902901e+09,2.326365e+09,2.365802e+08,6.964552e+08,1.186155e+09,1.542012e+08,9.050311e+08,9.427634e+08,1.619904e+08
1,2022-07-01,5.231306e+09,2.363314e+09,2.566823e+08,7.161922e+08,1.184074e+09,1.489671e+08,1.258915e+09,8.967837e+08,1.706272e+08
2,2022-08-01,4.768218e+09,2.307200e+09,2.406713e+08,6.817948e+08,1.171989e+09,1.539127e+08,9.505172e+08,8.158190e+08,1.484419e+08
3,2022-09-01,4.593704e+09,2.279238e+09,2.111380e+08,5.828307e+08,1.260664e+09,1.597025e+08,8.105768e+08,8.034217e+08,1.344185e+08
4,2022-10-01,5.240987e+09,2.404863e+09,2.583149e+08,6.312128e+08,1.294381e+09,1.577684e+08,1.174917e+09,1.007543e+09,1.636710e+08


DataFrame from sheet 'EVM value':


...

,Periods,POWDERED MILK,ALASKA | PLAIN,ALASKA | PLAIN | SACHET | 33 GM,ALASKA | PLAIN | BAG | 300 GM,ALASKA | PLAIN | BAG | 330 GM,ALASKA | PLAIN | BAG | 350 GM,BEAR BRAND | FILLED MILK,BIRCH TREE | FILLED MILK
0,2022-06-01,1.136784e+10,7.950207e+08,1.668680e+08,1.601237e+08,1.678974e+07,NaN,7.301089e+09,2.972880e+09
1,2022-07-01,1.178398e+10,8.427649e+08,1.697938e+08,1.762563e+08,1.734494e+07,NaN,7.501304e+09,3.096218e+09
2,2022-08-01,1.186124e+10,8.206418e+08,1.731508e+08,1.500686e+08,2.002756e+07,NaN,7.688553e+09,3.022613e+09
3,2022-09-01,1.140832e+10,8.525838e+08,1.584230e+08,1.622501e+08,1.603036e+07,NaN,7.213132e+09,3.049121e+09
4,2022-10-01,1.147940e+10,7.923038e+08,1.529363e+08,1.598295e+08,1.147038e+07,NaN,7.161243e+09,3.233013e+09


DataFrame from sheet 'RTD value':


,Periods,READY-TO-DRINK MILK,RTD Milk Less Plant based,RTD Milk Less Plant based | ALASKA MILK,RTD Milk Less Plant based | ALASKA MILK | ADULT,RTD Milk Less Plant based | ALASKA MILK | KIDS
0,2022-06-01,1.757617e+09,1.519581e+09,4.371379e+07,3.193887e+07,1.177492e+07
1,2022-07-01,1.775124e+09,1.521830e+09,5.171812e+07,3.624407e+07,1.547405e+07
2,2022-08-01,1.799410e+09,1.548943e+09,5.215430e+07,3.405492e+07,1.809938e+07
3,2022-09-01,1.840666e+09,1.578863e+09,5.376659e+07,2.863045e+07,2.513614e+07
4,2022-10-01,1.890243e+09,1.628063e+09,5.554908e+07,2.382180e+07,3.172729e+07


DataFrame from sheet 'RTD volume':


,Periods,READY-TO-DRINK MILK,RTD Milk Less Plant based,RTD Milk Less Plant based | ALASKA MILK,RTD Milk Less Plant based | ALASKA MILK | ADULT,RTD Milk Less Plant based | ALASKA MILK | KIDS
0,2022-06-01,1.747545e+10,1.519658e+10,4.876761e+08,3.815274e+08,1.061487e+08
1,2022-07-01,1.738898e+10,1.496599e+10,5.656829e+08,4.272014e+08,1.384815e+08
2,2022-08-01,1.735117e+10,1.497822e+10,5.538033e+08,3.935603e+08,1.602430e+08
3,2022-09-01,1.716900e+10,1.471733e+10,5.450380e+08,3.249215e+08,2.201165e+08
4,2022-10-01,1.749919e+10,1.507534e+10,5.336594e+08,2.632156e+08,2.704438e+08


## Identify time series and target variables

### Subtask:
For each dataframe (representing a tab), identify the 'Periods' column as the time index and all other numeric columns as individual target variables to be forecasted.


**Reasoning**:
Iterate through the dataframes, identify the 'Periods' column and other numeric columns as target variables, store the target columns, and print the identified columns for verification.



In [10]:
target_variables = {}

for sheet_name, df in dataframes.items():
    time_index_column = 'Periods'
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    target_cols = [col for col in numeric_cols if col != time_index_column]

    target_variables[sheet_name] = target_cols

    print(f"DataFrame: {sheet_name}")
    print(f"Time Index Column: {time_index_column}")
    print(f"Target Variable Columns: {target_cols}")
    print("-" * 30)

DataFrame: SCM value
Time Index Column: Periods
Target Variable Columns: ['CONDENSED MILK', 'ALASKA MILK', 'ALASKA | REGULAR', 'ALASKA | CONDENSADA', 'COWBELL', 'LIBERTY', 'ETIKA DAIRES', 'LTH FOOD INDUSTRIES', 'CENTURY PACIFIC FOOD']
------------------------------
DataFrame: SCM volume
Time Index Column: Periods
Target Variable Columns: ['CONDENSED MILK', 'ALASKA MILK', 'ALASKA | REGULAR', 'ALASKA | CONDENSADA', 'COWBELL', 'LIBERTY', 'ETIKA DAIRES', 'LTH FOOD INDUSTRIES', 'CENTURY PACIFIC FOOD']
------------------------------
DataFrame: EVM value
Time Index Column: Periods
Target Variable Columns: ['EVAPORATED MILK', 'Total AMC', 'Total Alaska Regular', 'Total Alaska Evaporada', 'Total Cowbell', 'Total Alpine']
------------------------------
DataFrame: EVM volume
Time Index Column: Periods
Target Variable Columns: ['EVAPORATED MILK', 'Total AMC', 'Total Alaska Regular', 'Total Alaska Evaporada', 'Total Cowbell', 'Total Alpine']
------------------------------
DataFrame: Creams value
Ti

## Data preprocessing

### Subtask:
Ensure the 'Periods' column is in datetime format and set as the index. Handle any missing values if necessary (I will check for this during data loading and preprocessing).


**Reasoning**:
Iterate through the dataframes, convert the 'Periods' column to datetime, set it as the index, and handle missing values using interpolation.



In [23]:
from itertools import product
import warnings
warnings.filterwarnings("ignore") # Suppress warnings from statsmodels

def calculate_mape(actual, forecast):
    """Calculates Mean Absolute Percentage Error (MAPE)."""
    # Avoid division by zero by replacing zeros with a small number or handling them
    # Here, we'll add a small epsilon to the actual values to avoid division by zero.
    # A more robust approach might involve handling zeros differently depending on the context.
    epsilon = 1e-10
    return np.mean(np.abs((actual - forecast) / (actual + epsilon))) * 100

def evaluate_model(model, train_data, test_data=None):
    """Evaluates a time series model and returns evaluation metrics."""
    results = {}
    try:
        # Fit the model
        if hasattr(model, 'fit'):
            model_fit = model.fit()
        else: # This case should not be reached after removing Prophet
             model_fit = None # Placeholder


        # In-sample predictions for AIC/BIC if test_data is None
        if test_data is None:
             if hasattr(model_fit, 'aic'):
                 results['AIC'] = model_fit.aic
             if hasattr(model_fit, 'bic'):
                 results['BIC'] = model_fit.bic
             # No MAPE calculation without a test set

        else: # If test_data is available (though not in this specific case based on user feedback)
            # Make predictions on the test set
            if hasattr(model_fit, 'predict'):
                 forecast = model_fit.predict(start=test_data.index[0], end=test_data.index[-1])
            # The Prophet specific forecast part is removed


            # Calculate MAPE
            results['MAPE'] = calculate_mape(test_data, forecast)

    except Exception as e:
        results['error'] = str(e)
        print(f"Error evaluating model: {e}")

    return results


def generate_forecast(model, train_data, forecast_horizon):
    """Generates future forecasts using the fitted model."""
    try:
        if hasattr(model, 'fit'):
            model_fit = model.fit()
        else: # This case should not be reached after removing Prophet
             model_fit = None # Placeholder


        if hasattr(model_fit, 'predict'):
            # Generate future dates for forecasting
            last_date = train_data.index[-1]
            future_dates = pd.date_range(start=last_date, periods=forecast_horizon + 1, freq='MS')[1:] # Assuming Monthly Start frequency

            forecast = model_fit.predict(start=future_dates[0], end=future_dates[-1])
            forecast.index = future_dates
        # The Prophet specific forecast part is removed


        return forecast

    except Exception as e:
        print(f"Error generating forecast: {e}")
        return None


# Define the forecast horizon
forecast_horizon = 60

# Define the ARIMA parameter grid
d = [0, 1]
p = [0, 1, 2, 3, 4]
q = [0, 1, 2, 3, 4]
seasonal_order_grid = [(0, 0, 0, 12), (0, 1, 0, 12), (1, 0, 1, 12)]
arima_param_combinations = list(product(p, d, q, seasonal_order_grid))

# Dictionary to store best models and forecasts
best_models = {}
all_forecasts = {}
model_performance = {}

# Iterate through each sheet and target variable
for sheet_name, targets in target_variables.items():
    best_models[sheet_name] = {}
    all_forecasts[sheet_name] = {}
    model_performance[sheet_name] = {}

    for target_col in targets:
        print(f"Forecasting for Sheet: {sheet_name}, Target: {target_col}")

        # Get the training data (full dataset in this case)
        train_data = dataframes[sheet_name][target_col].dropna()

        if len(train_data) == 0:
            print(f"No data available for {target_col} in {sheet_name}. Skipping.")
            print("-" * 30)
            continue

        # Store performance for each model for this time series
        model_performance[sheet_name][target_col] = {}

        # --- ARIMA Model with Grid Search ---
        print("Evaluating ARIMA models...")
        best_aic = np.inf
        best_arima_params = None
        best_seasonal_order = None

        for params in arima_param_combinations:
            order = params[:3]
            seasonal_order = params[3]
            try:
                # Fit SARIMAX model
                model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
                model_fit = model.fit()

                # Use AIC for model selection
                if model_fit.aic is not None and model_fit.aic < best_aic:
                    best_aic = model_fit.aic
                    best_arima_params = order
                    best_seasonal_order = seasonal_order

            except Exception as e:
                # print(f"ARIMA params {order}x{seasonal_order} failed: {e}")
                continue # Skip to the next combination if fitting fails

        if best_arima_params:
            print(f"Best ARIMA parameters for {target_col}: Order={best_arima_params}, Seasonal Order={best_seasonal_order}, AIC={best_aic:.2f}")
            # Fit the best ARIMA model
            best_arima_model = SARIMAX(train_data, order=best_arima_params, seasonal_order=best_seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
            model_performance[sheet_name][target_col]['ARIMA'] = {'AIC': best_aic}
            best_models[sheet_name]['ARIMA'] = best_arima_model # Store the unfitted model instance

        else:
            print(f"Could not find a suitable ARIMA model for {target_col}.")
            model_performance[sheet_name][target_col]['ARIMA'] = {'error': 'No suitable model found'}


        # --- Other Common Time Series Models (reduced to 2) ---
        print("Evaluating other common models (Holt-Winters)...")


        # Holt-Winters Exponential Smoothing (considering seasonality)
        try:
            hw_model = ExponentialSmoothing(train_data, seasonal='add', seasonal_periods=12) # Assuming additive seasonality and 12 periods
            hw_results = evaluate_model(hw_model, train_data)
            model_performance[sheet_name][target_col]['HoltWinters'] = hw_results
            best_models[sheet_name]['HoltWinters'] = hw_model # Store the unfitted model instance

        except Exception as e:
             model_performance[sheet_name][target_col]['HoltWinters'] = {'error': str(e)}
             print(f"Holt-Winters failed for {target_col}: {e}")


        # Prophet Model - REMOVED


        # --- Select Best Model and Generate Forecast ---
        print("Selecting best model and generating forecast...")

        # Determine the best model based on available metrics (prioritizing AIC for ARIMA)
        best_model_name = None
        best_model_perf_metric = np.inf # Using AIC as the primary criterion for comparison where available

        # Check ARIMA's AIC
        if 'ARIMA' in model_performance[sheet_name][target_col] and 'AIC' in model_performance[sheet_name][target_col]['ARIMA']:
             best_model_perf_metric = model_performance[sheet_name][target_col]['ARIMA']['AIC']
             best_model_name = 'ARIMA'

        # Check Holt-Winters AIC (if available)
        if 'HoltWinters' in model_performance[sheet_name][target_col] and 'AIC' in model_performance[sheet_name][target_col]['HoltWinters']:
             if model_performance[sheet_name][target_col]['HoltWinters']['AIC'] < best_model_perf_metric:
                  best_model_perf_metric = model_performance[sheet_name][target_col]['HoltWinters']['AIC']
                  best_model_name = 'HoltWinters'

        # Prophet is removed, so we compare ARIMA and Holt-Winters based on AIC

        if best_model_name:
             print(f"Best model selected for {target_col}: {best_model_name}")
             # Generate forecast using the best model instance
             best_model_instance = best_models[sheet_name][best_model_name]
             forecast = generate_forecast(best_model_instance, train_data, forecast_horizon)

             if forecast is not None:
                 all_forecasts[sheet_name][target_col] = forecast
                 print(f"Generated {forecast_horizon}-period forecast for {target_col}.")
             else:
                 print(f"Failed to generate forecast for {target_col}.")
        else:
             print(f"Could not select a best model for {target_col} based on available criteria.")


        print("-" * 50)

# Display model performance summary
print("\nModel Performance Summary (based on AIC/BIC where applicable):")
for sheet_name, targets in model_performance.items():
    print(f"\nSheet: {sheet_name}")
    for target_col, performance in targets.items():
        print(f"  Target: {target_col}")
        for model_name, metrics in performance.items():
            print(f"    {model_name}: {metrics}")

# Display generated forecasts (all 60 periods)
print("\nGenerated Forecasts (all 60 periods):")
for sheet_name, forecasts in all_forecasts.items():
    print(f"\nSheet: {sheet_name}")
    for target_col, forecast_data in forecasts.items():
        print(f"  Target: {target_col}")
        if forecast_data is not None:
            display(forecast_data) # Display the entire forecast data
        else:
            print("    Forecast not generated due to model selection or error.")

Forecasting for Sheet: SCM value, Target: CONDENSED MILK
Evaluating ARIMA models...
Best ARIMA parameters for CONDENSED MILK: Order=(1, 1, 4), Seasonal Order=(0, 1, 0, 12), AIC=767.79
Evaluating other common models (Holt-Winters)...
Selecting best model and generating forecast...
Best model selected for CONDENSED MILK: ARIMA
Generated 60-period forecast for CONDENSED MILK.
--------------------------------------------------
Forecasting for Sheet: SCM value, Target: ALASKA MILK
Evaluating ARIMA models...
Best ARIMA parameters for ALASKA MILK: Order=(2, 1, 4), Seasonal Order=(0, 1, 0, 12), AIC=737.60
Evaluating other common models (Holt-Winters)...
Selecting best model and generating forecast...
Best model selected for ALASKA MILK: ARIMA
Generated 60-period forecast for ALASKA MILK.
--------------------------------------------------
Forecasting for Sheet: SCM value, Target: ALASKA | REGULAR
Evaluating ARIMA models...
Best ARIMA parameters for ALASKA | REGULAR: Order=(0, 1, 4), Seasonal Or

,predicted_mean
2025-08-01,7.287292e+08
2025-09-01,7.347283e+08
2025-10-01,6.917288e+08
2025-11-01,7.738906e+08
2025-12-01,1.179381e+09
2026-01-01,1.391798e+09
2026-02-01,6.592669e+08
2026-03-01,8.037951e+08
2026-04-01,8.190958e+08
2026-05-01,9.782522e+08


  Target: ALASKA MILK


,predicted_mean
2025-08-01,4.319468e+08
2025-09-01,4.312326e+08
2025-10-01,4.522168e+08
2025-11-01,4.845359e+08
2025-12-01,7.805956e+08
2026-01-01,8.481349e+08
2026-02-01,4.375184e+08
2026-03-01,4.959309e+08
2026-04-01,5.249532e+08
2026-05-01,5.697879e+08


  Target: ALASKA | REGULAR


...

,predicted_mean
2025-08-01,5.383641e+08
2025-09-01,4.923307e+08
2025-10-01,4.925915e+08
2025-11-01,4.190698e+08
2025-12-01,4.520557e+08
2026-01-01,4.593115e+08
2026-02-01,4.771832e+08
2026-03-01,5.824721e+08
2026-04-01,5.206572e+08
2026-05-01,5.054062e+08


  Target: RTD Milk Less Plant based | ALASKA MILK | ADULT


,predicted_mean
2025-08-01,2.733266e+08
2025-09-01,2.457196e+08
2025-10-01,2.370169e+08
2025-11-01,2.296723e+08
2025-12-01,2.706451e+08
2026-01-01,2.613239e+08
2026-02-01,2.449771e+08
2026-03-01,2.847654e+08
2026-04-01,2.634323e+08
2026-05-01,2.782077e+08


  Target: RTD Milk Less Plant based | ALASKA MILK | KIDS


,predicted_mean
2025-08-01,2.710995e+08
2025-09-01,2.745561e+08
2025-10-01,2.417650e+08
2025-11-01,1.929040e+08
2025-12-01,1.964666e+08
2026-01-01,2.393193e+08
2026-02-01,2.805213e+08
2026-03-01,2.850380e+08
2026-04-01,2.290104e+08
2026-05-01,1.797075e+08


**Reasoning**:
The error indicates that the 'Periods' column is not found. This is likely because it was already set as the index in a previous step. I need to check the index name instead of trying to access a column named 'Periods'.



## Final Findings Summary

This section summarizes the results of the time series forecasting for each numeric column across all sheets in the uploaded Excel file.

For each time series, the best model was selected primarily based on AIC (Akaike Information Criterion) for ARIMA and Holt-Winters models where available. Due to the limited number of data points (38), a standard train-test split for evaluating models using MAPE on unseen data was not feasible.

The 60-period forecasts generated by the selected best model for each time series are also presented below.

---

In [18]:
print("--- Best Model Selection and Performance Summary ---")
for sheet_name, targets in model_performance.items():
    print(f"\nSheet: **{sheet_name}**")
    for target_col, performance in targets.items():
        print(f"  Target: **{target_col}**")
        best_model_info = "No best model selected based on criteria."
        best_model_name = None
        min_criterion_value = np.inf

        # Check ARIMA's AIC for best model selection
        if 'ARIMA' in performance and 'AIC' in performance['ARIMA']:
             min_criterion_value = performance['ARIMA']['AIC']
             best_model_name = 'ARIMA'
             best_model_info = f"    Best Model: {best_model_name} (AIC: {performance['ARIMA']['AIC']:.2f})"

        # Check Holt-Winters AIC (if available) for best model selection
        if 'HoltWinters' in performance and 'AIC' in performance['HoltWinters']:
             if performance['HoltWinters']['AIC'] is not None and performance['HoltWinters']['AIC'] < min_criterion_value:
                  min_criterion_value = performance['HoltWinters']['AIC']
                  best_model_name = 'HoltWinters'
                  best_model_info = f"    Best Model: {best_model_name} (AIC: {performance['HoltWinters']['AIC']:.2f})"

        # If neither ARIMA nor Holt-Winters were selected based on AIC, check if Prophet was successful
        if best_model_name is None and 'Prophet' in performance and 'error' not in performance['Prophet']:
             best_model_name = 'Prophet'
             best_model_info = f"    Best Model: {best_model_name} (Selected as other models were not optimal or successful based on AIC)"

        print(best_model_info)


        # Print performance for all evaluated models for this target
        print("    Evaluated Model Performance:")
        for model_name, metrics in performance.items():
            print(f"      {model_name}: {metrics}")

        print("-" * 30)


print("\n--- Generated Forecasts (all 60 periods for each time series) ---")
for sheet_name, forecasts in all_forecasts.items():
    print(f"\nSheet: **{sheet_name}**")
    for target_col, forecast_data in forecasts.items():
        print(f"  Target: **{target_col}**")
        if forecast_data is not None:
            display(forecast_data) # Display the entire forecast data
        else:
            print("    Forecast not generated due to model selection or error.")

print("\n--- End of Findings ---")

--- Best Model Selection and Performance Summary ---

Sheet: **SCM value**
  Target: **CONDENSED MILK**
    Best Model: ARIMA (AIC: 767.79)
    Evaluated Model Performance:
      ARIMA: {'AIC': 767.7889394581356}
      HoltWinters: {'AIC': 1351.1156284933431, 'BIC': 1374.0418347295126}
      Prophet: {'error': "Prophet.fit() missing 1 required positional argument: 'df'"}
------------------------------
  Target: **ALASKA MILK**
    Best Model: ARIMA (AIC: 737.60)
    Evaluated Model Performance:
      ARIMA: {'AIC': 737.6033016499732}
      HoltWinters: {'AIC': 1301.0353073950334, 'BIC': 1323.961513631203}
      Prophet: {'error': "Prophet.fit() missing 1 required positional argument: 'df'"}
------------------------------
  Target: **ALASKA | REGULAR**
No best model selected based on criteria.
    Evaluated Model Performance:
------------------------------

--- Generated Forecasts (all 60 periods for each time series) ---

Sheet: **SCM value**
  Target: **CONDENSED MILK**


,predicted_mean
2025-08-01,7.287292e+08
2025-09-01,7.347283e+08
2025-10-01,6.917288e+08
2025-11-01,7.738906e+08
2025-12-01,1.179381e+09
2026-01-01,1.391798e+09
2026-02-01,6.592669e+08
2026-03-01,8.037951e+08
2026-04-01,8.190958e+08
2026-05-01,9.782522e+08


  Target: **ALASKA MILK**


,predicted_mean
2025-08-01,4.319468e+08
2025-09-01,4.312326e+08
2025-10-01,4.522168e+08
2025-11-01,4.845359e+08
2025-12-01,7.805956e+08
2026-01-01,8.481349e+08
2026-02-01,4.375184e+08
2026-03-01,4.959309e+08
2026-04-01,5.249532e+08
2026-05-01,5.697879e+08



--- End of Findings ---


## Export Results to Excel

### Subtask:
Export the model performance summary and generated forecasts to an Excel file.

In [24]:
output_excel_path = '/content/time_series_forecast_results.xlsx'

# Create a Pandas Excel writer using XlsxWriter as the engine.
with pd.ExcelWriter(output_excel_path, engine='xlsxwriter') as writer:

    # Write the Model Performance Summary to a sheet
    summary_data = []
    for sheet_name, targets in model_performance.items():
        for target_col, performance in targets.items():
            best_model_info = "No best model selected based on criteria."
            best_model_name = None
            min_criterion_value = np.inf

            # Determine the best model based on stored information
            if sheet_name in best_models and target_col in best_models[sheet_name]:
                 # Check ARIMA's AIC
                 if 'ARIMA' in performance and 'AIC' in performance['ARIMA']:
                     min_criterion_value = performance['ARIMA']['AIC']
                     best_model_name = 'ARIMA'
                     best_model_info = f"Best Model: {best_model_name} (AIC: {performance['ARIMA']['AIC']:.2f})"

                 # Check Holt-Winters AIC (if available)
                 if 'HoltWinters' in performance and 'AIC' in performance['HoltWinters']:
                     if performance['HoltWinters']['AIC'] is not None and performance['HoltWinters']['AIC'] < min_criterion_value:
                          min_criterion_value = performance['HoltWinters']['AIC']
                          best_model_name = 'HoltWinters'
                          best_model_info = f"Best Model: {best_model_name} (AIC: {performance['HoltWinters']['AIC']:.2f})"

                 # If neither ARIMA nor Holt-Winters were selected based on AIC, check if Prophet was successful
                 if best_model_name is None and 'Prophet' in performance and 'error' not in performance['Prophet']:
                      best_model_name = 'Prophet'
                      best_model_info = f"Best Model: {best_model_name} (Selected as other models were not optimal or successful based on AIC)"


            # Collect performance metrics for all models
            performance_details = "; ".join([f"{model_name}: {metrics}" for model_name, metrics in performance.items()])

            summary_data.append([sheet_name, target_col, best_model_info, performance_details])

    summary_df = pd.DataFrame(summary_data, columns=['Sheet', 'Target Variable', 'Best Model Summary', 'Evaluated Model Performance'])
    summary_df.to_excel(writer, sheet_name='Model Summary', index=False)

    # Write the Generated Forecasts to separate sheets or sections
    for sheet_name, forecasts in all_forecasts.items():
        forecast_sheet_name = f"{sheet_name}_Forecasts"
        startrow = 0
        for target_col, forecast_data in forecasts.items():
            if forecast_data is not None:
                # Add a header for the target variable
                header_df = pd.DataFrame([f"Forecast for: {target_col}"])
                header_df.to_excel(writer, sheet_name=forecast_sheet_name, startrow=startrow, index=False, header=False)
                startrow += 1 # Move to the next row for the data

                # Write the forecast data
                # Convert Series to DataFrame for easier writing, keeping the datetime index
                forecast_df = forecast_data.reset_index()
                forecast_df.columns = ['Period', 'Forecast Value']
                forecast_df.to_excel(writer, sheet_name=forecast_sheet_name, startrow=startrow, index=False)
                startrow += len(forecast_df) + 2 # Move to the next row, add some space

            else:
                # Indicate that forecast was not generated
                no_forecast_df = pd.DataFrame([f"Forecast for: {target_col}", "Forecast not generated due to model selection or error."])
                no_forecast_df.to_excel(writer, sheet_name=forecast_sheet_name, startrow=startrow, index=False, header=False)
                startrow += len(no_forecast_df) + 2 # Move to the next row, add some space


print(f"\nResults exported successfully to '{output_excel_path}'")


Results exported successfully to '/content/time_series_forecast_results.xlsx'


In [20]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.5 MB/s eta 0:00:00
